## DeepEval

### Imports and logging in

In [4]:
from deepeval.models.base_model import DeepEvalBaseLLM
from huggingface_hub import login
import os
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig

load_dotenv()
login(os.getenv("HUGGINGFACE_API_KEY"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\filip\.cache\huggingface\token
Login successful


### Loading Llama-3 8B

In [5]:
checkpoint = "meta-llama/Meta-Llama-3-8B-Instruct"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    device_map="auto",
    # torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### Testing loaded model

In [12]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Arrrr, shiver me timbers! Me name be Captain Chatbot, the scourge o' the seven seas... o' text! I be a swashbucklin' bot, here to chat with ye about all manner o' things, from the finest booty to the most treacherous waters. So hoist the colors, me hearty, and let's set sail fer a chat!


In [13]:
messages = [
    {"role": "user", "content": "Generate very short JSON file."},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Here is a very short JSON file:

```
{
  "name": "John",
  "age": 30
}
```


### Class for DeepEval

In [14]:
class CustomLlama3_8B(DeepEvalBaseLLM):
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.terminators = [
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        messages = [
            {"role": "user", "content": prompt},
        ]
        
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)
        
        outputs = model.generate(
            input_ids,
            max_new_tokens=256,
            eos_token_id=self.terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
        )
        response = outputs[0][input_ids.shape[-1]:]
        
        return tokenizer.decode(response, skip_special_tokens=True)

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return "Llama-3 8B"

In [15]:
custom_LLM = CustomLlama3_8B(model, tokenizer)

In [17]:
print(custom_LLM.generate("Tell a joke."))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Why don't scientists trust atoms?

Because they make up everything!


### DeepEval metrics
#### Hallucination

In [21]:
from deepeval import evaluate
from deepeval.metrics import HallucinationMetric
from deepeval.test_case import LLMTestCase

# Replace this with the actual documents that you are passing as input to your LLM.
context=["A man with blond-hair, and a brown shirt drinking out of a public water fountain."]

# Replace this with the actual output from your LLM application
actual_output="A blond drinking water in public."

test_case = LLMTestCase(
    input="What was the blond doing?",
    actual_output=actual_output,
    context=context
)
metric = HallucinationMetric(threshold=0.5, model=custom_LLM)

metric.measure(test_case)
print(metric.score)
print(metric.reason)

# or evaluate test cases in bulk
evaluate([test_case], [metric])

Output()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Output()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


0.0
The score is 0.00 because the actual output perfectly aligns with the provided context, indicating no hallucination.
Evaluating test cases...
Event loop is already running. Applying nest_asyncio patch to allow async execution...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.




Metrics Summary

  - ✅ Hallucination (score: 0.0, threshold: 0.5, strict: False, evaluation model: Llama-3 8B, reason: The score is 0.00 because the actual output does not contain any information that contradicts the provided context, which mentions a blond-hair person., error: None)

For test case:

  - input: What was the blond doing?
  - actual output: A blond drinking water in public.
  - expected output: None
  - context: ['A man with blond-hair, and a brown shirt drinking out of a public water fountain.']
  - retrieval context: None


Overall Metric Pass Rates

Hallucination: 100.00% pass rate



✅ Tests finished! Run "deepeval login" to view evaluation results on the web.

[TestResult(success=True, metrics_metadata=[MetricMetadata(metric='Hallucination', threshold=0.5, success=True, score=0.0, reason='The score is 0.00 because the actual output does not contain any information that contradicts the provided context, which mentions a blond-hair person.', strict_mode=False, evaluation_model='Llama-3 8B', error=None, evaluation_cost=None, verbose_logs='Verdicts:\n[\n    {\n        "verdict": "yes",\n        "reason": "The actual output agrees with the provided context which mentions a blond-hair person."\n    }\n]')], input='What was the blond doing?', actual_output='A blond drinking water in public.', expected_output=None, context=['A man with blond-hair, and a brown shirt drinking out of a public water fountain.'], retrieval_context=None)]